<a href="https://colab.research.google.com/github/amirpaia/blenderbot/blob/main/blender_90m_400m_multitask_double_sided.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.Installing prerequisites

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jun 16 07:43:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mydrive_path = '/content/drive/MyDrive/colabs/blender-models/'
# !pip uninstall -q parlai
!pip install -q parlai
!pip install transformers

In [ ]:
# !cp -v /content/drive/MyDrive/colabs/blender-models/finetuned-reddit-90m--finetuned-4tasks-5epochs/* /content/drive/MyDrive/colabs/blender-models/finetuned-reddit-90m/

In [ ]:
!parlai eval_model --task blended_skill_talk --model-file zoo:blender/blender_90M/model --batchsize 32

# 1.Preparing Data

## Genreal Functions

In [ ]:
def transfer_list_of_turns_to_dialog(d):
    if len(d)%2 !=0: d = d[:-1]
    t = ""
    for i in range(0,len(d),2):
        u1 = d[i]
        u2 = d[i+1]

        if (i+2) != len(d):
            t += "text:"+u1+"\t"+"labels:"+u2+"\n"
        else:
            t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"
    return t

def transfer_list_of_pairs_to_dialog(d):
  t = ""
  for i, text_label_pair in enumerate(d):
    u1 = text_label_pair[0]
    u2 = text_label_pair[1]

    if i != (len(d) - 1):
      t += "text:"+u1+"\t"+"labels:"+u2+"\n"
    else:
      t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"

  return t

def convert_parlai_format_to_list_of_turns(lines):
    result = []
    for line in lines:
        text_label = line.split("\t")
        result.append(text_label[0].replace("text:", ""))
        result.append(text_label[1].replace("labels:", "").replace("\n",""))
    return result

t = ['hello','how are you','good','bye','test']
print(transfer_list_of_turns_to_dialog(t))

t = [['hello','how are you'],['good','bye']]
print(transfer_list_of_pairs_to_dialog(t))

text:hello	labels:how are you
text:good	labels:bye	episode_done:True

text:hello	labels:how are you
text:good	labels:bye	episode_done:True



In [ ]:
def make_dataset_double_sided(lines, path, filename): 
    # train[:10]
    all_dialogs_parlai_format = []
    dialog = []
    for line in lines:
        dialog.append(line)
        if 'episode_done:True' in line:
            turns = convert_parlai_format_to_list_of_turns(dialog)
            first_parlai_dialog = transfer_list_of_turns_to_dialog(turns)
            second_parlai_dialog = transfer_list_of_turns_to_dialog(turns[1:])

            all_dialogs_parlai_format.append(first_parlai_dialog)
            all_dialogs_parlai_format.append(second_parlai_dialog)

            dialog = []
            # break
    print(sum([1 for a in lines if 'episode_done:True' in a]), len(all_dialogs_parlai_format))

    with open(f"{path}{filename}", "w") as f:
        f.writelines(all_dialogs_parlai_format)

# !mkdir /content/dataset_french_bst/
# make_dataset_double_sided(lines,"/content/dataset_french_bst/", "train.txt")



# # import os.path
# # from os import path

## XPersona

In [ ]:
!git clone https://github.com/HLTCHKUST/Xpersona.git

import json
import pandas as pd

with open('Xpersona/dataset/Fr_persona_train_corrected.json','r') as f:
   train_data = json.load(f)

dialogs_train = pd.DataFrame(train_data)['dialogue'].tolist()

with open('Xpersona/dataset/Fr_persona_split_valid_human_annotated.json','r') as f:
   valid_data = json.load(f)

dialogs_valid = pd.DataFrame(valid_data)['dialogue'].tolist()

with open('Xpersona/dataset/Fr_persona_split_test_human_annotated.json','r') as f:
   test_data = json.load(f)

dialogs_test = pd.DataFrame(test_data)['dialogue'].tolist()

# dialogs

Cloning into 'Xpersona'...
remote: Enumerating objects: 285, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 285 (delta 6), reused 6 (delta 4), pack-reused 275
Receiving objects: 100% (285/285), 45.01 MiB | 19.31 MiB/s, done.
Resolving deltas: 100% (96/96), done.


In [ ]:
data_path = "/content/dataset_french_xpersona/"
!rm -R $data_path
!mkdir $data_path

def convert_xpersona_dialogs_to_parlai_format_file_and_double_sided(dialogs, filename):
    all_dialogs_parlai_format = []
    import numpy as np
    for d in dialogs:
        turns = np.reshape(d, (-1)).tolist()
        all_dialogs_parlai_format.append(transfer_list_of_turns_to_dialog(turns))
        all_dialogs_parlai_format.append(transfer_list_of_turns_to_dialog(turns[1:]))

    print(len(all_dialogs_parlai_format))

    with open(f"{data_path}{filename}","w") as f:
        f.writelines(all_dialogs_parlai_format)

convert_xpersona_dialogs_to_parlai_format_file_and_double_sided(dialogs_train, "train.txt")
convert_xpersona_dialogs_to_parlai_format_file_and_double_sided(dialogs_valid, "valid.txt")
convert_xpersona_dialogs_to_parlai_format_file_and_double_sided(dialogs_test, "test.txt")

rm: cannot remove '/content/dataset_french_xpersona/': No such file or directory
33756
496
498


In [ ]:
# data_path = "/content/dataset_french_xpersona/"
# !rm -R $data_path
# !mkdir $data_path

# #region Training
# data_train = ""
# for d in dialogs_train:
#   data_train += transfer_list_of_pairs_to_dialog(d)

# file_train = open(f"{data_path}train.txt","w")
# print("Training Set:", file_train.write(data_train))
# #endregion 
    
# #region Validation
# data_valid = ""
# for d in dialogs_valid:
#   data_valid += transfer_list_of_pairs_to_dialog(d)

# file_valid = open(f"{data_path}valid.txt","w")
# print("Validation Set:", file_valid.write(data_valid))
# #endregion

# #region Test
# data_test = ""
# for d in dialogs_test:
#   data_test += transfer_list_of_pairs_to_dialog(d)

# file_test = open(f"{data_path}test.txt","w")
# print("Test Set:", file_test.write(data_test))
# #endregion 

# # print(len(data_valid))

## ED

In [ ]:
googledrive_data_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_empathetic_dialogues/"
data_path = "/content/dataset_french_ed/"
!rm -R $data_path
!mkdir $data_path
!cp -rv $googledrive_data_path* $data_path

rm: cannot remove '/content/dataset_french_ed/': No such file or directory
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_empathetic_dialogues/test.txt' -> '/content/dataset_french_ed/test.txt'
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_empathetic_dialogues/train.txt' -> '/content/dataset_french_ed/train.txt'
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_empathetic_dialogues/valid.txt' -> '/content/dataset_french_ed/valid.txt'


In [ ]:
# with open(f"{googledrive_data_path}train.txt") as f:
#     train = f.readlines()

# with open(f"{googledrive_data_path}valid.txt") as f:
#     valid = f.readlines()

# with open(f"{googledrive_data_path}test.txt") as f:
#     test = f.readlines()


# print(len(train))
# a, b, c = sum([1 for a in train if 'episode_done:True' in a]), sum([1 for a in valid if 'episode_done:True' in a]), sum([1 for a in test if 'episode_done:True' in a])
# print(a,b,c,a+b+c)


64636
39057 5537 5093 49687


## BST

In [ ]:
googledrive_data_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/"
data_path = "/content/dataset_french_bst/"
!rm -R $data_path
!mkdir $data_path
!cp -rv $googledrive_data_path* $data_path

rm: cannot remove '/content/dataset_french_bst/': No such file or directory
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/test.txt' -> '/content/dataset_french_bst/test.txt'
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/train.txt' -> '/content/dataset_french_bst/train.txt'
'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/valid.txt' -> '/content/dataset_french_bst/valid.txt'


In [ ]:
googledrive_data_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_blended_skill_talk/"
with open(f"{googledrive_data_path}train.txt") as f:
    train = f.readlines()

with open(f"{googledrive_data_path}valid.txt") as f:
    valid = f.readlines()

with open(f"{googledrive_data_path}test.txt") as f:
    test = f.readlines()


print(len(train))
a, b, c = sum([1 for a in train if 'episode_done:True' in a]), sum([1 for a in valid if 'episode_done:True' in a]), sum([1 for a in test if 'episode_done:True' in a])
print(a, b, c, a+b+c)




59700
9638 2018 1958 13614


## WoW

In [ ]:
# googledrive_data_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_wizard_of_wikipedia/"
# data_path = "/content/dataset_french_wow/"
# !rm -R $data_path
# !mkdir $data_path
# !cp -rv $googledrive_data_path* $data_path

In [ ]:
# convert_xpersona_dialogs_to_parlai_format_file_and_double_sided(dialogs_train, "train.txt")
googledrive_data_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_wizard_of_wikipedia/"
data_path = "/content/dataset_french_wow/"
!rm -R $data_path
!mkdir $data_path

with open(f"{googledrive_data_path}train.txt") as f:
    train_lines = f.readlines()
with open(f"{googledrive_data_path}valid.txt") as f:
    valid_lines = f.readlines()
with open(f"{googledrive_data_path}test.txt") as f:
    test_lines = f.readlines()


make_dataset_double_sided(train_lines, data_path, "train.txt")
make_dataset_double_sided(valid_lines, data_path, "valid.txt")
make_dataset_double_sided(test_lines, data_path, "test.txt")

rm: cannot remove '/content/dataset_french_wow/': No such file or directory
17848 35696
2231 4462
2231 4462


In [ ]:
# googledrive_data_path = "/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_wizard_of_wikipedia/"
# with open(f"{googledrive_data_path}train.txt") as f:
#     train = f.readlines()
# with open(f"{googledrive_data_path}valid.txt") as f:
#     valid = f.readlines()
# with open(f"{googledrive_data_path}test.txt") as f:
#     test = f.readlines()
# print(len(train))
# a, b, c = sum([1 for a in train if 'episode_done:True' in a]), sum([1 for a in valid if 'episode_done:True' in a]), sum([1 for a in test if 'episode_done:True' in a])
# print(a, b, c, a+b+c)

# 2.Creating new Tasks

In [ ]:
#region XPersona
task_path = "/usr/local/lib/python3.7/dist-packages/parlai/tasks/"

!rm -R $task_path'french_xpersona/'
!mkdir $task_path'french_xpersona'
!cp -ruv /content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_xpersona/* $task_path'french_xpersona/'
#endregion


#region ED
task_path = "/usr/local/lib/python3.7/dist-packages/parlai/tasks/"

!rm -R $task_path'french_empathetic_dialogues/'
!mkdir $task_path'french_empathetic_dialogues'
!cp -ruv /content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_empathetic_dialogues/* $task_path'french_empathetic_dialogues/'
#endregion


#region BST
task_path = "/usr/local/lib/python3.7/dist-packages/parlai/tasks/"

!rm -R $task_path'french_blended_skill_talk/'
!mkdir $task_path'french_blended_skill_talk'
!cp -ruv /content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_blended_skill_talk/* $task_path'french_blended_skill_talk/'
#endregion

#region WoW
task_path = "/usr/local/lib/python3.7/dist-packages/parlai/tasks/"

!rm -R $task_path'french_wizard_of_wikipedia/'
!mkdir $task_path'french_wizard_of_wikipedia'
!cp -ruv /content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_wizard_of_wikipedia/* $task_path'french_wizard_of_wikipedia/'
#endregion 

rm: cannot remove '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_xpersona/': No such file or directory
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_xpersona/agents.py' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_xpersona/agents.py'
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_xpersona/build.py' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_xpersona/build.py'
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_xpersona/__init__.py' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_xpersona/__init__.py'
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_xpersona/LICENSE_DOCUMENTATION' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_xpersona/LICENSE_DOCUMENTATION'
'/content/drive/MyDrive/colabs/aliae-workspace/parlai-tasks/french_xpersona/README.md' -> '/usr/local/lib/python3.7/dist-packages/parlai/tasks/french_xpersona/README.md'
'/conten

In [ ]:
# !parlai display_data --task french_empathetic_dialogues
# !parlai display_data --task french_xpersona 
# !parlai display_data --task french_blended_skill_talk 
# !parlai display_data --task french_wizard_of_wikipedia

# 3.Finetuning + Multitasking

In [ ]:
# finetuned_model_path = f'{mydrive_path}finetuned-reddit-90m--finetuned-4tasks-5epochs/'
finetuned_model_path = f'{mydrive_path}finetuned-reddit_LELU-4tasks-90m'
# init_model = 'zoo:blender/blender_400Mdistill/model'
# dict_file  = 'zoo:blender/blender_400Mdistill/model.dict'
init_model = 'zoo:blender/blender_90M/model'
dict_file  = 'zoo:blender/blender_90M/model.dict'
finetuned_model_path
# from os import path
# path.exists(f"{finetuned_model_path}model")

'/content/drive/MyDrive/colabs/blender-models/finetuned-reddit-90m--finetuned-4tasks-5epochs/'

In [ ]:
# !mkdir copied_dataset_french_bst
# !cp dataset_french_bst/* copied_dataset_french_bst
# !mv copied_dataset_french_bst/test.txt copied_dataset_french_bst/_test.txt
# !mv copied_dataset_french_bst/train.txt copied_dataset_french_bst/_train.txt
# !mv copied_dataset_french_bst/valid.txt copied_dataset_french_bst/_valid.txt

In [ ]:
!pip install wandb

In [ ]:
# 90M settings
# !rm -rf $finetuned_model_path
!mkdir -p $finetuned_model_path


from parlai.scripts.train_model import TrainModel

TrainModel.main(
    # task
    task= "french_blended_skill_talk,french_wizard_of_wikipedia,french_xpersona,french_empathetic_dialogues",
    multitask_weights= "1,3,3,3",

    # task='fromfile:parlaiformat', 
    # fromfile_datapath= f'{data_path}data',
    # fromfile_datatype_extension=True,

    model='transformer/generator',
    model_file= f'{finetuned_model_path}model',
    
    # initialize with a pretrained model
    init_model= init_model,
    dict_file=dict_file,
    
    # arguments we get from the pretrained model.
    # Unfortunately, these must be looked up separately for each model.
    n_heads=16, n_layers=8, n_positions=512, text_truncate=512,
    label_truncate=128, ffn_size=2048, embedding_size=512,
    activation='gelu', variant='xlm',
    dict_lower=True, dict_tokenizer='bpe',
    
    # depend on your gpu. 
    
    validation_every_n_epochs=0.1,
    num_epochs = 10,
    log_every_n_secs= 180,
    verbose = True,
    batchsize= 8, 
    fp16= True, fp16_impl= "mem_efficient",
    save_after_valid = True,
    wandb_log = True,
    
    # arguments we get from the pretrained model.
    
    # speeds up validation
    skip_generation=True,
    vp= 10,
    validation_metric= "ppl", #vmt = "ppl"
    validation_metric_mode= "min", # vmm= "min"
    
    # helps us cram more examples into our gpu at a time
    dynamic_batching='full',

    
    # some training arguments, specific to this fine-tuning
    lr=1e-5, optimizer='adam',
    attention_dropout= 0.0, 
    model_parallel= False,
    warmup_updates=100,

    # customized parameters
    # inference= "beam"
    # beam_min_length= 20,
    # beam_block_ngram= 3,
    # beam_context_block_ngram= 3,
    # beam_size= 10,
)

14:21:20 | building dictionary first...
14:21:20 | Overriding opt["multitask_weights"] to (1.0, 3.0, 3.0, 3.0) (previously: [1.0, 3.0, 3.0, 3.0])
14:21:20 | Overriding opt["init_model"] to zoo:blender/blender_90M/model (previously: /content/drive/MyDrive/colabs/blender-models/finetuned-reddit-90m--finetuned-4tasks-5epochs/model.checkpoint)
14:21:20 | Overriding opt["dict_file"] to /usr/local/lib/python3.7/dist-packages/data/models/blender/blender_90M/model.dict (previously: /content/drive/MyDrive/colabs/blender-models/finetuned-reddit-90m--finetuned-4tasks-5epochs/model.checkpoint.dict)
14:21:20 | Overriding opt["validation_every_n_epochs"] to 0.1 (previously: 0.25)
14:21:20 | Overriding opt["optimizer"] to adam (previously: mem_eff_adam)
14:21:20 | your model is being loaded with opts that do not exist in the model you are initializing the weights with: verbose: True,download_path: None,datapath: /usr/local/lib/python3.7/dist-packages/data,load_from_checkpoint: True,interactive_mode: 

14:21:52 | training...
14:22:03 | time:43096s total_exs:4191034 total_steps:200030 epochs:8.41 time_left:8176s
                                clen  clip  ctpb  ctps  ctrunc  ctrunclen  exps  exs  fp16_loss_scalar  gnorm  gpu_mem  llen  \
   all                         55.32     1  2357 11796       0          0 225.8 2256             16384  5.905    .4126 28.03   
   french_blended_skill_talk   64.21                         0          0        184                                   29.52   
   french_empathetic_dialogues 41.84                         0          0        735                                   26.95   
   french_wizard_of_wikipedia  63.08                         0          0        507                                   33.78   
   french_xpersona             52.14                         0          0        830                                   21.88   
                                loss        lr  ltpb  ltps   ltrunc  ltrunclen   ppl  token_acc  token_em  \
   all      

clen/train,▁▅▆█▆▇▆▇▇▆▆▇▇▆▇▆▆█▆▆▆▇▇▇▆▇▇▇▆▇▇▇▆▆▆▇▆▇▇
clen/valid,▁
clip/train,█████▁█████████████████████████████████
ctpb/train,▁▆▆▇▅▆▅██▅▇▆▅▇▅▅▅▇▇▆▆▇▆▆▆▅▆▇▅▆▆▅▆▄▄▇▄▅▇
ctpb/valid,▁
ctps/train,▁▃▆▁▅▃▅▅█▁▇▃▅▃▅▂▅▄▇▂▇▃▆▃▇▂▇▃▅▃▆▂▆▁▄▃▄▃█
ctps/valid,▁
ctrunc/train,▁▁▁▁▁▁▁▁▁█▁▂▁▁▁▁▁▁▁▁▅▁▃▄▁▁▁▆▃▁▃▄▃▁▁▂▁▂▁
ctrunc/valid,▁
ctrunclen/train,▁▁▁▁▁▁▁▁▁▇▁▄▁▁▁▁▁▁▁▁█▁▃▇▁▁▁█▄▁▂▃▄▁▁▁▁▂▁
ctrunclen/valid,▁


({'clen': MacroAverageMetric(144.9),
  'ctpb': GlobalAverageMetric(3432),
  'ctps': GlobalTimerMetric(5.339e+04),
  'ctrunc': MacroAverageMetric(0.004689),
  'ctrunclen': MacroAverageMetric(0.2258),
  'exps': GlobalTimerMetric(268.5),
  'exs': SumMetric(4.326e+04),
  'french_blended_skill_talk/clen': AverageMetric(199.5),
  'french_blended_skill_talk/ctrunc': AverageMetric(0.01395),
  'french_blended_skill_talk/ctrunclen': AverageMetric(0.612),
  'french_blended_skill_talk/exs': SumMetric(1.248e+04),
  'french_blended_skill_talk/llen': AverageMetric(30.98),
  'french_blended_skill_talk/loss': AverageMetric(2.172),
  'french_blended_skill_talk/ltrunc': AverageMetric(8.015e-05),
  'french_blended_skill_talk/ltrunclen': AverageMetric(0.0005611),
  'french_blended_skill_talk/ppl': PPLMetric(8.779),
  'french_blended_skill_talk/token_acc': AverageMetric(0.5431),
  'french_blended_skill_talk/token_em': AverageMetric(0),
  'french_empathetic_dialogues/clen': AverageMetric(60.69),
  'french_em

In [ ]:
# mydrive_path = '/content/finetuned-multitask-400m-double-sided-2epochs'
# mydrive_path = '/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-double-sided/'

In [ ]:
# 400M settings
# !rm -rf $finetuned_model_path
!mkdir -p $finetuned_model_path


from parlai.scripts.train_model import TrainModel

TrainModel.main(
    # task
    task= "french_blended_skill_talk,french_wizard_of_wikipedia,french_xpersona,french_empathetic_dialogues",
    multitask_weights= "1,3,3,3",

    # task='fromfile:parlaiformat', 
    # fromfile_datapath= f'{data_path}data',
    # fromfile_datatype_extension=True,

    model='transformer/generator',
    model_file= f'{finetuned_model_path}model',
    
    # initialize with a pretrained model
    init_model= init_model,
    dict_file=dict_file,
    
    # depend on your gpu
    validation_every_n_epochs=0.25, # veps= 0.25, 
    num_epochs = 10,
    log_every_n_secs= 300,
    verbose = True,
    attention_dropout= 0.0, 
    batchsize= 16, 
    fp16= True, fp16_impl= "mem_efficient",
    # save_after_valid= True,

    # arguments we get from the pretrained model. "from recipes page for 2.7B model" 
    embedding_size= 1280, ffn_size= 5120,
    variant= "prelayernorm",
    n_heads= 32, n_positions= 128, 
    n_encoder_layers= 2, n_decoder_layers= 12,

    label_truncate= 128, text_truncate= 128, truncate= 128,
    activation= "gelu",
    history_add_global_end_token= "end", 
    delimiter= '  ', 
    dict_tokenizer= "bytelevelbpe",
    dropout= 0.1,
    
    # some training arguments, specific to this fine-tuning
    lr= 7e-06, lr_scheduler= "reduceonplateau", lr_scheduler_patience= 3,
    optimizer= "mem_eff_adam",
    relu_dropout= 0.0, 
    model_parallel= False,
    warmup_updates= 100,
    update_freq= 2,
    gradient_clip= 0.1, 

    # speeds up validation
    skip_generation= True,
    vp= 10,
    validation_metric= "ppl", #vmt = "ppl"
    validation_metric_mode= "min", # vmm= "min"

    # customized parameters
    # inference = 'topk', 
    # temperature = 0.7, 
    # topk=30, 
    # beam_length_penalty=1.03
)

08:17:05 | building dictionary first...
08:17:05 | Overriding opt["multitask_weights"] to (1.0, 3.0, 3.0, 3.0) (previously: [1.0, 3.0, 3.0, 3.0])
08:17:05 | Overriding opt["dict_file"] to /usr/local/lib/python3.7/dist-packages/data/models/blender/blender_90M/model.dict (previously: /content/drive/MyDrive/colabs/blender-models/finetuned-reddit-90m--finetuned-4tasks-5epochs/model.dict)
08:17:05 | Overriding opt["num_epochs"] to 10.0 (previously: 5.0)
08:17:05 | Overriding opt["log_every_n_secs"] to 300.0 (previously: 180.0)
08:17:05 | Overriding opt["batchsize"] to 16 (previously: 8)
08:17:05 | Overriding opt["save_after_valid"] to True (previously: False)
08:17:05 | Overriding opt["embedding_size"] to 1280 (previously: 512)
08:17:05 | Overriding opt["ffn_size"] to 5120 (previously: 2048)
08:17:05 | Overriding opt["variant"] to prelayernorm (previously: xlm)
08:17:05 | Overriding opt["n_heads"] to 32 (previously: 16)
08:17:05 | Overriding opt["n_positions"] to 128 (previously: 512)
08:17

OSError: ignored

In [ ]:
# !cp -rv /content/finetuned-multitask-400m-double-sided-2epochs/* /content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-double-sided/
# !ls -lah /content/finetuned-multitask-400m-double-sided-2epochs/
# !ls -lah /content/finetuned-multitask-400m-double-sided/

# 4.Display Model

In [ ]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='french_blended_skill_talk',
    model_file= f'{finetuned_model_path}/model',
    num_examples=20,
    skip_generation=False,
)

In [ ]:
finetuned_model_path = f'{mydrive_path}finetuned-multitask-90m/'
finetuned_model_path

'/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-90m/'

In [ ]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='french_blended_skill_talk',
    # task='fromfile:parlaiformat', 
    # fromfile_datapath= f'{data_path}data',
    # fromfile_datatype_extension=True,
    datatype= "test",

    model_file= f'{finetuned_model_path}model',
    num_examples=20,
    skip_generation=False,

    # the result of grid search on 400M model and BST dataset when inference=topk
    beam_block_ngram= 2,
	beam_context_block_ngram= 3,
	beam_length_penalty= 1,
	beam_min_length= 10,
	beam_size= 20,
	inference= "topk",
	temperature= 0.5,
	topk= 20,
	topp= 0.9

    # # Farnaz sent me
    # beam_block_ngram= 3,
    # beam_context_block_ngram= 3,
    # beam_min_length= 20, 
    # beam_size= 10,
    # inference =  'topk',  
    # topk=20, 
    # temperature = 0.5, 
    # beam_length_penalty=0.8
)

14:42:28 | Overriding opt["task"] to french_blended_skill_talk (previously: french_blended_skill_talk,french_xpersona,french_empathetic_dialogues,french_wizard_of_wikipedia)
14:42:28 | Overriding opt["datatype"] to test (previously: train)
14:42:28 | Overriding opt["skip_generation"] to False (previously: True)
14:42:28 | Overriding opt["beam_block_ngram"] to 2 (previously: -1)
14:42:28 | Overriding opt["beam_context_block_ngram"] to 3 (previously: -1)
14:42:28 | Overriding opt["beam_length_penalty"] to 1.0 (previously: 0.65)
14:42:28 | Overriding opt["beam_min_length"] to 10 (previously: 1)
14:42:28 | Overriding opt["beam_size"] to 20 (previously: 1)
14:42:28 | Overriding opt["inference"] to topk (previously: greedy)
14:42:28 | Overriding opt["temperature"] to 0.5 (previously: 1.0)
14:42:28 | Overriding opt["topk"] to 20 (previously: 10)
14:42:28 | Using CUDA
14:42:28 | loading dictionary from /content/drive/MyDrive/colabs/blender-models/finetuned-multitask-90m/model.dict
14:42:28 | n

# 5.Display Data

In [ ]:
from parlai.scripts.display_data import DisplayData
DisplayData.main(
    task='blended_skill_talk',
    num_examples=50
)

09:51:55 | Opt:
09:51:55 |     allow_missing_init_opts: False
09:51:55 |     batchsize: 1
09:51:55 |     datapath: /usr/local/lib/python3.7/dist-packages/data
09:51:55 |     datatype: train:ordered
09:51:55 |     dict_class: None
09:51:55 |     display_add_fields: 
09:51:55 |     download_path: None
09:51:55 |     dynamic_batching: None
09:51:55 |     hide_labels: False
09:51:55 |     ignore_agent_reply: True
09:51:55 |     image_cropsize: 224
09:51:55 |     image_mode: raw
09:51:55 |     image_size: 256
09:51:55 |     init_model: None
09:51:55 |     init_opt: None
09:51:55 |     is_debug: False
09:51:55 |     loglevel: info
09:51:55 |     max_display_len: 1000
09:51:55 |     model: None
09:51:55 |     model_file: None
09:51:55 |     multitask_weights: [1]
09:51:55 |     mutators: None
09:51:55 |     num_examples: 50
09:51:55 |     override: "{'task': 'blended_skill_talk', 'num_examples': 50}"
09:51:55 |     parlai_home: /usr/local/lib/python3.7/dist-packages
09:51:55 |     starttime: 